In [1]:
import time
import random
from collections import deque


In [3]:
class Server:
    def __init__(self, server_id):
        self.server_id = server_id
        self.busy = False
        self.tasks_handled = 0
        self.total_busy_time = 0

    def assign_request(self, request, current_time):
        self.busy = True
        self.finish_time = current_time + request['processing_time']
        self.tasks_handled += 1
        self.total_busy_time += request['processing_time']
        print(f"Server {self.server_id} assigned Request {request['id']} (Arrival: {request['arrival_time']}, Processing: {request['processing_time']}) at time {current_time}")

    def update_status(self, current_time):
        if self.busy and current_time >= self.finish_time:
            self.busy = False

In [5]:
class LoadBalancer:
    def __init__(self, servers):
        self.servers = servers
        self.queue = deque()
        self.current_index = 0
        self.total_waiting_time = 0
        self.total_requests = 0
        self.request_id_counter = 0

    def add_request(self, arrival_time, processing_time):
        request = {
            'id': self.request_id_counter,
            'arrival_time': arrival_time,
            'processing_time': processing_time
        }
        self.queue.append(request)
        self.total_requests += 1
        self.request_id_counter += 1
        print(f"Request {request['id']} added at time {arrival_time} with processing time {processing_time}")

    def assign_requests(self, current_time):
        while self.queue:
            assigned = False
            for _ in range(len(self.servers)):
                server = self.servers[self.current_index]
                server.update_status(current_time)
                if not server.busy:
                    request = self.queue.popleft()
                    waiting_time = max(0, current_time - request['arrival_time'])
                    self.total_waiting_time += waiting_time
                    server.assign_request(request, current_time)
                    assigned = True
                    break
                self.current_index = (self.current_index + 1) % len(self.servers)
            if not assigned:
                break  # No server was available

In [7]:
def simulate_round_robin(num_servers=3, num_requests=50, max_processing_time=5, simulation_duration=100):
    servers = [Server(i) for i in range(num_servers)]
    lb = LoadBalancer(servers)

    current_time = 0
    while current_time < simulation_duration:
        if num_requests > 0 and random.random() < 0.7:
            processing_time = random.randint(1, max_processing_time)
            lb.add_request(current_time, processing_time)
            num_requests -= 1

        lb.assign_requests(current_time)
        current_time += 1
        time.sleep(0.05)

    for server in servers:
        server.update_status(current_time)

    # Final report
    print("\n=== Simulation Report ===")
    print(f"Total Requests Processed: {lb.total_requests}")
    print(f"Average Waiting Time: {lb.total_waiting_time / lb.total_requests:.2f} time units")
    for server in servers:
        utilization = (server.total_busy_time / simulation_duration) * 100
        print(f"Server {server.server_id}:")
        print(f"  - Tasks Handled: {server.tasks_handled}")
        print(f"  - Utilization: {utilization:.2f}%")

if __name__ == "__main__":
    simulate_round_robin(num_servers=4, num_requests=100, max_processing_time=5, simulation_duration=60)


Request 0 added at time 2 with processing time 5
Server 0 assigned Request 0 (Arrival: 2, Processing: 5) at time 2
Request 1 added at time 3 with processing time 2
Server 1 assigned Request 1 (Arrival: 3, Processing: 2) at time 3
Request 2 added at time 4 with processing time 2
Server 2 assigned Request 2 (Arrival: 4, Processing: 2) at time 4
Request 3 added at time 5 with processing time 3
Server 3 assigned Request 3 (Arrival: 5, Processing: 3) at time 5
Request 4 added at time 7 with processing time 2
Server 0 assigned Request 4 (Arrival: 7, Processing: 2) at time 7
Request 5 added at time 8 with processing time 2
Server 1 assigned Request 5 (Arrival: 8, Processing: 2) at time 8
Request 6 added at time 11 with processing time 2
Server 1 assigned Request 6 (Arrival: 11, Processing: 2) at time 11
Request 7 added at time 12 with processing time 3
Server 2 assigned Request 7 (Arrival: 12, Processing: 3) at time 12
Request 8 added at time 14 with processing time 2
Server 3 assigned Reques